In [12]:
import requests
import json
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt


#extract data from Stock Vantage
response = requests.get('https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=aapl&outputsize=full&apikey=GZZHIBO1SGTLPGB1')
data = response.text
parsed = json.loads(data)
tsdata = parsed['Time Series (Daily)']
tsda = pd.DataFrame.from_dict(tsdata, orient='index')
tsda_new=tsda.loc[:'2009-01-02',]
tsda_new.index = pd.to_datetime(tsda_new.index)
tsda_new.index=tsda_new.index.map(datetime.datetime.toordinal)


In [13]:
#extracting price from tsda_new
price=tsda_new.drop(['1. open','2. high','3. low','4. close','6. volume','7. dividend amount','8. split coefficient'],1)
price.rename(columns={"5. adjusted close":"price"},inplace=True)
price.price = pd.to_numeric(price.price)
price.insert(1, "fed_interest",np.nan)
price.insert(2, "dividend_yield",np.nan)

In [14]:
#extracting dividends from tsda_new
dividend=tsda_new.drop(['1. open','2. high','3. low','5. adjusted close','6. volume','8. split coefficient'],1)
dividend.rename(columns={"4. close":"price","7. dividend amount":"adjusted_dividend"},inplace=True)
#cleaning of dividend data
dividend.adjusted_dividend = pd.to_numeric(dividend.adjusted_dividend)
dividend.price = pd.to_numeric(dividend.price)
#remove zeroes
indexNames = dividend[ dividend['adjusted_dividend'] == 0 ].index
dividend.drop(indexNames , inplace=True)
#account for stock split
dividend_before_2020=dividend.loc['2020-08-07':,]
dividend_before_2020=(dividend_before_2020)/4
dividend_before_2014=dividend_before_2020.loc['2014-05-08':,]
dividend_before_2014=dividend_before_2014/7
dividend.update(dividend_before_2020)
dividend.update(dividend_before_2014)
dividend['dividend_yield'] = dividend['adjusted_dividend']/dividend['price']*100

In [28]:
#extract interest rates
interest = pd.read_csv('FEDFUNDS.csv',index_col=0)
interest.index = pd.to_datetime(interest.index)
interest.index = interest.index.map(datetime.datetime.toordinal)
interest = interest.drop(interest.columns[1:], axis=1)

In [27]:
#adding variables to the dataframe
price.update(interest)
price.update(dividend)

#Adding data points for quarterly dividend yield
div=0
date=4417
i=price.shape[0]-1
for date in price.index:
    if pd.isnull(price.iloc[i,2]):
        price.iloc[i,2]=div
        i-=1
    else:
        div=price.iloc[i,2]
        i-=1

price.index.names = ['Date']
print(price)

             price  fed_interest  dividend_yield
Date                                            
737888  130.360000          0.07        0.149898
737887  127.900000          0.07        0.149898
737886  126.210000          0.07        0.149898
737885  125.900000          0.07        0.149898
737881  123.000000          0.07        0.149898
...            ...           ...             ...
733415    2.851361          0.15        0.000000
733414    2.799378          0.15        0.000000
733413    2.861204          0.15        0.000000
733412    2.909188          0.15        0.000000
733409    2.791381          0.15        0.000000

[3087 rows x 3 columns]
